In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast

import torchvision
import torchvision.transforms as transforms

In [2]:
torch.backends.cudnn.benchmark=True

In [3]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
train_data = torchvision.datasets.CIFAR10(root='./data', train = True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train = False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
image, label = train_data[0]

In [6]:
image.size()

torch.Size([3, 32, 32])

In [7]:
class_names=['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse','ship', 'truck']

In [8]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv1 =nn.Conv2d(3, 12, 5) #32-5=27/1=27+1= (12, 28, 28)
        self.pool = nn.MaxPool2d(2,2) #(12, 14, 14)
        self.conv2 = nn.Conv2d(12,24, 5) # 14-5 =9/1= 9+1= (24, 10, 10) ->(24, 5, 5)
        self.fc1 = nn.Linear(24*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x= torch.flatten(x, 1)
        x= F.relu(self.fc1(x))
        x= F.relu(self.fc2(x))
        x= self.fc3(x)
        return x

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net = NeuralNet().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001) #Adam Optimizer to try and train faster
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) #SGD

if torch.cuda.is_available():
    scaler = torch.cuda.amp.GradScaler()
else:
    scaler = None

In [12]:
for epoch in range(30):
    print (f'Training epoch {epoch}...')
    net.train()
    running_loss=0.0

    for i, data in enumerate(train_loader):
        inputs, labels=data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        if scaler:
            with torch.cuda.amp.autocast():
                output = net(inputs)
                loss = loss_function(output, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            output = net(inputs)
            loss = loss_function(output, labels)
            loss.backward()
            optimizer.step()


        running_loss+= loss.item()

    print(f'Loss: {running_loss/ len(train_loader):.4f}')

Training epoch 0...
Loss: 1.7727
Training epoch 1...
Loss: 1.5038
Training epoch 2...
Loss: 1.3873
Training epoch 3...
Loss: 1.3092
Training epoch 4...
Loss: 1.2463
Training epoch 5...
Loss: 1.1969
Training epoch 6...
Loss: 1.1546
Training epoch 7...
Loss: 1.1311
Training epoch 8...
Loss: 1.0949
Training epoch 9...
Loss: 1.0697
Training epoch 10...
Loss: 1.0517
Training epoch 11...


KeyboardInterrupt: 

In [13]:
torch.save(net.state_dict(), 'trained_net.pth')

In [14]:
net= NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))
net.to(device)

C:\Users\Jason\AppData\Local\Temp\ipykernel_24164\1738933767.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


NeuralNet(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=600, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [15]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct +=(predicted ==labels).sum().item()


accuracy = 100*correct/total
print(f'Accuracy: {accuracy}%')

Accuracy: 62.78%


In [16]:
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image= image.unsqueeze(0)
    return image

image_paths = ['example1.jpg', 'example2.jpg']
images = [load_image(img).to(device) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()]}')
        

Prediction: dog
Prediction: plane
